In [ ]:
import numpy as np
import tqdm
import pandas as pd

In [ ]:
def ThermoTXT_to_LongDF(filename):
    df_cols = ["scannumber", "time", "intensity", "m_over_z"]
    df = []
    
    n_scans = 0
    for line in open(filename, "r"):
        if "ScanHeader" in line:
            n_scans = int(line.split("#")[1].strip())
    print(f"There are {n_scans} rts")

    current_scan = 0
    for line in open(filename, "r"):
        if "ScanHeader" in line:
            current_scan = int(line.split("#")[1].strip())
            print(f"{current_scan}/{n_scans}")
        if "start_time" in line:
            time = str(line.split(", ")[0].strip())
            time = float(time.split("= ")[1].strip())
        if ' intensity =' in line:
            _, intensity, m_over_z = line.split(",")
            intensity = float(intensity.replace("intensity = ", "").strip())
            m_over_z = float(m_over_z.replace("mass/position = ", "").strip())
            df.append([
                current_scan,
                time,
                intensity,
                m_over_z
            ])
    df2 = pd.DataFrame(df, columns=df_cols)
    #df2.head(10)
    new_filename = "df_" + filename
    df2.to_csv(new_filename, header=df_cols, index=None, sep='\t', mode='a')

In [ ]:
#ThermoTXT_to_LongDF("19007sss05.txt")

In [ ]:
import os

In [ ]:
directory_list = os.listdir("C:/Users/Marilyn/Desktop/Thermo_convert")
filenames = []
for filename in directory_list:
    #print (filename) #all files, folders
    if ".txt" in filename:
        print (filename)
        filenames.append(filename)

In [ ]:
print(filenames)

In [ ]:
for filename in filenames:
    print(filename)
    ThermoTXT_to_LongDF(filename)

In [ ]:
## old code from 3S

In [ ]:
data = np.array(df)

In [ ]:
del df

In [ ]:
def pivot_table(data):
    rows, row_pos = np.unique(data[:, 0], return_inverse=True)
    cols, col_pos = np.unique(data[:, 1], return_inverse=True)

    pivot_table = np.zeros((len(rows), len(cols)), dtype=data.dtype)
    pivot_table[row_pos, col_pos] = data[:, 2]
    return pivot_table

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df = pd.DataFrame(data, columns=df_cols)

In [ ]:
df.m_over_z.unique()

In [ ]:
len(df[df.scannumber==1200].m_over_z.unique())

In [ ]:
len(df.m_over_z.unique())

In [ ]:
plt.figure()
plt.scatter(
    df[df.scannumber == 1600].m_over_z,
    np.log10(df[df.scannumber == 1600].intensity),
    marker='.'
)

In [ ]:
data_small = []
for scannumber in tqdm.tqdm(range(1, n_scans+1)):
    data_small.append(
        np.interp(
            x=np.arange(0, 800, 1),
            xp=df[df.scannumber == scannumber].m_over_z,
            fp=df[df.scannumber == scannumber].intensity
        )
    )

In [ ]:
data_small = np.array(data_small)

In [ ]:
plt.figure()
plt.contour(np.log10(data_small+1))
plt.colorbar()
plt.show()